<a href="https://colab.research.google.com/github/tortoisehare/TSR-GAN/blob/master/TSRGAN_dataprep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()


Saving test.p to test.p
Saving train.p to train.p


In [0]:
import pandas as pd
import numpy as np
#import torch
#import torch.nn as nn

In [0]:
# Load pickled traffic sign data
import pickle


training_file = 'train.p'
testing_file = 'test.p'

with open(training_file, mode='rb') as f:
    tstrain = pickle.load(f, encoding='bytes')
with open(testing_file, mode='rb') as f:
    tstest = pickle.load(f, encoding='bytes')


In [0]:
#convert to consolidated numpy array for later
#NOTE: pickled version of images have shape (n, 32, 32, 3)
#images denoted by dictionary name 'features'

traindata = np.array(tstrain['features'])
#print(traindata.shape)
traindata = traindata.T.reshape(39209,32*32*3)
#print(traindata.shape)

testdata = np.array(tstest['features'])
#print(testdata.shape)
testdata = testdata.T.reshape(12630,32*32*3)
#print(testdata.shape)

#pickled version of labels have shape (n,), dict name 'labels'
trainlabels = np.array(tstrain['labels'])
trainlabels = trainlabels.reshape(39209,1)
#print(trainlabels.shape)
testlabels = np.array(tstest['labels'])
testlabels = testlabels.reshape(12630,1)
#print(testlabels.shape)
#print(testlabels)

#add labels as column to end of data, check shape
tsrtrain = np.concatenate((traindata,trainlabels),axis=1)
tsrtest = np.concatenate((testdata,testlabels),axis=1)
#print(tsrtest.shape)

In [0]:
#load sign names csv

#url = 'https://raw.githubusercontent.com/tortoisehare/TSR-GAN/master/signnames.csv'
#dfs = pd.read_csv(url)

import csv

sign_names = []
with open('signnames.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        sign_names.append(row[1])
    sign_names.reverse()
    sign_names.pop()
    sign_names.reverse()

In [0]:
#load CIFAR10 dataset (one of the test minibatches, 10,000 random images)
cifarfile = 'data_batch_1.p'

with open(cifarfile, mode='rb') as fo:
  cifar = pickle.load(fo,encoding='bytes')

#data will be 10000x3072 numpy array, each row storing 32x32 colour image
#labels will be a list of 10,000 numbers in range 0-9

#make all labels 43
labels = np.full((10000,1), 43)
#print(labels)


#pull out numpy array of images
images = np.array(cifar[b'data']) #dict name has b due to bytes-encoding
#print(images)

#add labels as last column to dataset
cifartot = np.concatenate((images,labels),axis=1)
#print(cifartot)

#split into train set (800 images) and test set (400 images)
#dataset randomly ordered so will get different classes
#(though maybe not all ten)
cifartrain = cifartot[:800]
cifartest = cifartot[801:1200]
print(cifartrain.shape)


In [0]:
#merge CIFAR10 images with TSR images

traindata = np.concatenate((cifartrain,tsrtrain))
#print(traindata)
testdata = np.concatenate((cifartest,tsrtest))
#print(testdata)

#convert numpy arrays to dataframe, add column name to labels
dftrain = pd.DataFrame(traindata,index=None,columns=None)
dftrain.rename(index=str, columns = {3072:"Labels"}, inplace=True)
print(dftrain.shape)
#dftrain.head()

dftest = pd.DataFrame(testdata,index=None,columns=None)
dftest.rename(index=str, columns = {3072:"Labels"}, inplace=True)
print(dftest.shape)
dftest.head()


In [0]:
#only want class 1, 5, and 43 for the first attempt
smalltrain = dftrain.loc[(dftrain["Labels"]==1)|(dftrain["Labels"]==5)|(dftrain["Labels"]==43)]
smalltest = dftest.loc[(dftest["Labels"]==1)|(dftest["Labels"]==5)|(dftest["Labels"]==43)]
smalltrain.head(10)

#convert to 2D numpy array for use in classifiers
smalltrain = smalltrain.to_numpy()
smalltest = smalltest.to_numpy()
print(smalltrain.shape())
pritn(smalltest.shape())


In [0]:
#Reference cell
#example of how to download files to local directory

from google.colab import files
np.savetxt("traindata.csv", traindata, delimiter=",")

files.download('traindata.csv')

CIFAR10: 
This tech report (Chapter 3) describes the dataset and the methodology followed when collecting it in much greater detail. Please cite it if you intend to use this dataset. 

Learning Multiple Layers of Features from Tiny Images, Alex Krizhevsky, 2009.